In [ ]:
import sys
import json
import shutil
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# pd.set_option('display.max_rows', 20)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

In [ ]:
crimes = pd.read_csv('..//data/cleaned/crime-processed.csv')

crimes.head()

In [ ]:
crimes.isnull().mean()

In [ ]:
crimes.dtypes

In [ ]:
# AREA NAME
print('Number of unique divisions: {}'.format(crimes['AREA NAME'].nunique()))
crimes['AREA NAME'].unique()

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1,1,1)
crimes.groupby('AREA NAME').size().plot(kind='bar', ax=ax)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1,1,1)
crimes.loc[crimes.Year != 2020].groupby(['Year', 'AREA NAME']).size().unstack().plot(ax=ax)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1,1,1)
crimes.loc[crimes.Year != 2020].groupby('Year')['Arrested'].mean().plot()

In [ ]:
arrest_by_area_year = crimes.loc[crimes.Year != 2020].groupby(['Year', 'AREA NAME'])['Arrested'].mean().unstack()
arrest_by_area_year

In [ ]:
fig=plt.figure(figsize=(16,26))
fig.tight_layout()
columns = 3
rows = 7
for idx, col in enumerate(arrest_by_area_year.columns):
    fig.add_subplot(rows, columns, idx+1)
    plt.plot(arrest_by_area_year[col])
    plt.title('Arrest Rate in {}'.format(col))
plt.show()

In [ ]:
# Crime Description
# How to classify? 
# 1. Felony, misdemeanor, infraction
# 2. Theft, burglary, murder, etc.
print(len(crimes['Crm Cd Desc'].value_counts()))
crimes['Crm Cd Desc'].value_counts()

In [ ]:
petty = ['petty','violation','yield','dumping','pickpocket','replica firearms',]

mis = ['simple assault','misdeameanor','misdemeanor','shoplift','reckless driving','abortion','inciting','disperse',
       'beastiality','disrupt school', 'drugs, to a minor','threat','trespassing','lewd','brandish','battery with sex',
      'battery police','contempt of court','indecent','resisting arrest','bomb scare','peeping','prowler','$950 & under',
      'document worthless','motorized',]

fel = ['felony','aggravated assault','grand','train wreck','manslaughter','incest','lynching','bribery','robbery','rape','child',
      'sex,unlawful','arson','foreign object','extortion','dwelling','kidnap','assault with dead','$950.01 & over','traffick',
      'pimping','pandering','homicide','chld']

wob = ['dwoc','bigamy','theft','telephone property','conspiracy','burglary','discharge','stalk','oral copulation','throwing',
      'sodomy','computer','cruelty','false imprisonment','counterfeit','false police report','battery on a firefighter',
      'shots fired at moving','weapons possession','stolen','snatch']

wobblette = ['disturbing',]

def clean_crm_cd_desc(x):
    lowered = x.lower()
    if any(substring in lowered for substring in petty):
        return 'Infraction'
    elif any(substring in lowered for substring in mis):
        return 'Misdemeanor'
    elif any(substring in lowered for substring in fel):
        return 'Felony'
    elif any(substring in lowered for substring in wob):
        return 'Wobbler'
    elif any(substring in lowered for substring in wobblette):
        return 'Wobblette'
    else:
        return 'Others'

In [ ]:
crimes['Crm Cd Desc'].apply(clean_crm_cd_desc).value_counts()

In [ ]:
crimes['Penalty'] = crimes['Crm Cd Desc'].apply(clean_crm_cd_desc)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1,1,1)
crimes.groupby('Penalty').size().plot(kind='bar',ax=ax)

In [ ]:
foo = crimes.groupby(['AREA NAME','Penalty']).size().unstack()
foo

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1,1,1)
stacked = plt.stackplot(foo.iloc[:11].index, foo.iloc[:11].T)
legendProxies = []
for stack in stacked:
    legendProxies.append(plt.Rectangle((0, 0), 1, 1, fc=stack.get_facecolor()[0]))
plt.legend(legendProxies, foo.columns)

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1,1,1)
stacked = plt.stackplot(foo.iloc[11:].index, foo.iloc[11:].T)
legendProxies = []
for stack in stacked:
    legendProxies.append(plt.Rectangle((0, 0), 1, 1, fc=stack.get_facecolor()[0]))
plt.legend(legendProxies, foo.columns)